In [2]:
import numpy as np

def read_tsv_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        data = []
        for line in lines[1:]:
            # Remove newline character and split values by tabs
            values = line.strip().split('\t')
            
            #skip the first column with names
            values = values[1:]
            
            # Convert values to floats and append to data list
            data.append([float(value) for value in values])
        # Convert data list to NumPy array
        array = np.array(data)
        return array

In [3]:
file_path = 'group_T1w.tsv'
data = read_tsv_file(file_path)

In [4]:
from tsne import bh_sne
data_tsne = bh_sne(data)
xs = data_tsne[:, 0]
ys = data_tsne[:, 1]

In [7]:
data=np.column_stack((data,xs,ys))

In [8]:
data.shape

(1134, 70)

In [9]:
from tsne import bh_sne
import numpy as np

In [ ]:
def mriqc2mrqy(input_file, output_file):
    with open(input_file, 'r') as f_input:
        lines = f_input.readlines()

    header = lines[0].strip().split('\t')
    header[0] = "#dataset:Patient"
    header.insert(1, "Name of Images")
    
    #add t-sne (and umap header)
    header.extend(["x", "y"])    

    with open(output_file, 'w') as f_output:
        f_output.write("#start_time:	2021-12-20 09:29:49.078558\n")
        f_output.write("#outdir:	/home/UserInterface/Data/GRIEG\n")
        
        f_output.write('\t'.join(header) +'\n')

        for line in lines[1:]:
            values = line.strip().split('\t')
            new_value = f"['{values[0]}.png']"
            values.insert(1, new_value)
            f_output.write('\t'.join(values) + '\n')

In [27]:
mriqc2mrqy('group_T1w.tsv','mod.tsv')

In [26]:
def mriqc2mrqy(input_file, output_file):
    with open(input_file, 'r') as f_input:
        lines = f_input.readlines()

    header = lines[0].strip().split('\t')
    header[0] = "#dataset:Patient"
    header.insert(1, "Name of Images")
    
    #add t-sne (and umap header)
    header.extend(["x", "y"])    

    with open(output_file, 'w') as f_output:
        f_output.write("#start_time:	2021-12-20 09:29:49.078558\n")
        f_output.write("#outdir:	/home/UserInterface/Data/GRIEG\n")
        
        f_output.write('\t'.join(header) +'\n')

        data=[]
        subjids=[]
        for line in lines[1:]:
            values = line.strip().split('\t')
            #values = values[1:] #skip first with subjids
            subjids.append(values[0])
            data.append([float(value) for value in values[1:]])
         
        #calculate tsne coordinates
        array = np.array(data)
        data_tsne = bh_sne(array)
        xs = data_tsne[:, 0]
        ys = data_tsne[:, 1]
        #data=np.column_stack((data,xs,ys))
        
        for subjid, values, x, y in zip(subjids,data,xs,ys):
            images = f"['{subjid}.png']"
            values.insert(0,images)
            values.insert(0,subjid)
            values.append(str(x))
            values.append(str(y))
            f_output.write('\t'.join(map(str,values)) + '\n')

In [59]:
import re
import base64

def extract_base64_image_from_html(html_file, output_file):
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Use regular expression to find the base64-encoded image data
    #base64_images = re.findall(r'xlink:href="data:image/(\w+);base64,(.*?)"', html_content)
    base64_images = re.findall(r'xlink:href="data:image/(\w+);base64,([^"]+)"', html_content)

    for i, (image_format, base64_data) in enumerate(base64_images):
        print(image_format)
        # Convert base64 data to bytes
        image_bytes = base64.b64decode(base64_data)

        # Save the image bytes to a new image file
        with open(f"{output_file}_{i}.{image_format}", 'wb') as f_output:
            f_output.write(image_bytes)






In [62]:
# Example usage:
html_file = 'sub-91025908_T1w.html'
output_file = 'extracted_image'
extract_images_and_stop(html_file, output_file)

/var/folders/yt/fj2qpycs3bb9n4yq8bry3wcm0000gn/T/ipykernel_1159/1968747242.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  stop_element = soup.find('div', class_='card-header', text='Background noise')


In [61]:
import re
import base64
from bs4 import BeautifulSoup

def extract_images_and_stop(html_file, output_file):
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    soup = BeautifulSoup(html_content, 'html.parser')
    img_tags = soup.find_all('img')

    stop_element = soup.find('div', class_='card-header', text='Background noise')
    stop_extracting = False
    image_counter = 0

    for i, img_tag in enumerate(img_tags):
        if stop_extracting:
            break

        src_data = img_tag.get('xlink:href')  # Check for xlink:href attribute
        if src_data and src_data.startswith('data:image/'):
            image_counter += 1
            if image_counter % 3 == 0:
                image_format = re.findall(r'data:image/(\w+);', src_data)[0]
                base64_data = re.sub('^data:image/\w+;base64,', '', src_data)
                image_bytes = base64.b64decode(base64_data)

                with open(f"{output_file}_{image_counter}.{image_format}", 'wb') as f_output:
                    f_output.write(image_bytes)

        # Check if the current tag is the stop element
        if img_tag == stop_element:
            stop_extracting = True
